In [1]:
!pip install -U pycoingecko
!pip install DeFiLlama
import matplotlib.pyplot as plt
import requests
import datetime as dt
import pandas as pd
import numpy as np
from google.colab import files
!pip install Tokenterminal
from tokenterminal import TokenTerminal
token_terminal = TokenTerminal(key='6c37f0ff-5c2b-4564-8286-3bccf6e42fd2')
import seaborn as sns
!cp /content/drive/MyDrive/Colab\ Notebooks/CoinGeckoFunctions_2.py /content
!cp /content/drive/MyDrive/Colab\ Notebooks/FundamentalFunctions.py /content
!cp /content/drive/MyDrive/Colab\ Notebooks/DefiLlamaFunctions.py /content
!cp /content/drive/MyDrive/Colab\ Notebooks/TokenTerminal_Functions_2.py /content

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import internal libraries

import TokenTerminal_Functions_2 as tt
import CoinGeckoFunctions_2 as cgf
import FundamentalFunctions as ff
import DefiLlamaFunctions as dl

In [6]:
# Historical data for PERPETUAL-PROTOCOL

perpetual = token_terminal.get_historical_metrics('perpetual-protocol')
PRICE_perpetual = tt.get_price(perpetual)
PRICE_perpetual['RETURN'] = (PRICE_perpetual['PRICE']/PRICE_perpetual['PRICE'].shift(1)) - 1
PRICE_perpetual['RETURN'] = PRICE_perpetual['RETURN']
PRICE_perpetual = PRICE_perpetual['2021-10-21':'2022-10-20']
MCAP_perpetual = tt.get_mcap_circ(perpetual)
MCAP_perpetual = MCAP_perpetual['2021-10-21':'2022-10-20']
MARKET_CAP_FULLY_DILUTED_perpetual = tt.get_mcap_fully_diluted(perpetual)
MARKET_CAP_FULLY_DILUTED_perpetual = MARKET_CAP_FULLY_DILUTED_perpetual['2021-10-21':'2022-10-20']
perp_volume = pd.DataFrame(pd.read_csv('/content/Daily Trading Volume In The Past 365 Days. 2022-10-21 perpetual.csv'))
perp_volume = perp_volume.set_index('Date')

project = {}
project = {'fee': 0.1/100, 'revenue share': 0.1/6/100, 'chain': 'xDAI', 'volume data': 'coingecko', 'tt_id': 'perpetual', 'tvl data': 'defillama', 'defillama slug': 'perpetual-protocol'}

def get_info():
    
    return project

def get_data():
    
    xyz = ff.VFR (perp_volume, project['fee'], project['revenue share'])
    zyx = dl.get_tvl(project['defillama slug'])
    zyx.columns = ['TVL']
    return xyz, zyx

data, tvl = get_data()
tvl = tvl[130:495]
tvl_ = list(tvl['TVL'])
data['TVL'] = tvl_

returns_perpetual_protocol = (PRICE_dydx['PRICE']/PRICE_dydx['PRICE'].shift(1)) - 1

def multiples(data, name, PRICE_perpetual, MCAP_perpetual, MARKET_CAP_FULLY_DILUTED_perpetual, current_price, market_cap, returns, days = 30):
    
    volume, fees, revenue, tvl = data['Volume'], data['Fees'], data['Revenue'], data['TVL']
    
    #volume = volume[name]
    #fees = fees[name]
    #revenue = revenue[name]
    
    tvl = tvl.replace('NaN', np.nan)
    #tvl = tvl[name]
    
    daily_avg_volume = volume[-days:].mean()
    annualized_volume = daily_avg_volume * 365 
    annualized_fees = fees[-days:].mean() * 365
    annualized_revenue = revenue[-days:].mean() * 365
    

    tvl_current = tvl[np.isnan(tvl)!=True][-1]   
    tvl_turnover = volume[-days:].sum() / tvl[-days:].mean()

    
    price = current_price
    std = np.std(PRICE_perpetual['RETURN'][-days:]) * np.sqrt(365) * 100
    MC = MCAP_perpetual[-1:]['market_cap_circulating'][0]    
    FDV = MARKET_CAP_FULLY_DILUTED_perpetual[-1:]['market_cap_fully_diluted'][0]
    MC_AF = MC / annualized_fees
    FDV_AF = FDV / annualized_fees
    MC_TVL = MC / tvl_current
    FDV_TVL = FDV / tvl_current
    
    
    
    if annualized_revenue!=0:
        MC_AR = MC / annualized_revenue
        FDV_AR = FDV / annualized_revenue
    
    else:
        MC_AR = np.nan
        FDV_AR = np.nan
    
    titles = ['Daily Average Volume', 'Annualized Volume', 'Annualized Fees (AF)',
         'Annualized Revenue (AR)', 'TVL', 'TVL Turnover ' + str(days) + ' days', 'Token Price', 'Realized Volatility %', 'Market Cap (MC)', 'Fully Diluted Valuation (FDV)',
              'MC / AF', 'FDV / AF', 'MC / AR', 'FDV / AR', 
              'MC / TVL', 'FDV / TVL']
            
    data = [daily_avg_volume, annualized_volume, annualized_fees, 
            annualized_revenue, tvl_current, tvl_turnover, price, std, MC, FDV, 
            MC_AF, FDV_AF, MC_AR, FDV_AR, MC_TVL, FDV_TVL]
    
    return pd.DataFrame(data = data, index = titles, columns=[name])

p = multiples(data, 'perpetual-protocol', PRICE_perpetual, MCAP_perpetual, MARKET_CAP_FULLY_DILUTED_perpetual, 0.55, 38600000, PRICE_perpetual['RETURN'], days = 30)

In [4]:
# Historical data for DYDX, GMX, MCDEX

dydx = token_terminal.get_historical_metrics('dydx')
PRICE_dydx = tt.get_price(dydx)
gmx = token_terminal.get_historical_metrics('gmx')
PRICE_gmx = tt.get_price(gmx)
mcdex = token_terminal.get_historical_metrics('mcdex')
PRICE_mcdex = tt.get_price(mcdex)

PRICE_dydx['RETURN'] = (PRICE_dydx['PRICE']/PRICE_dydx['PRICE'].shift(1)) - 1
PRICE_dydx['RETURN'] = PRICE_dydx['RETURN']
PRICE_dydx = PRICE_dydx['2021-10-21':'2022-10-20']

PRICE_gmx['RETURN'] = (PRICE_gmx['PRICE']/PRICE_gmx['PRICE'].shift(1)) - 1
PRICE_gmx['RETURN'] = PRICE_gmx['RETURN']
PRICE_gmx = PRICE_gmx['2021-10-21':'2022-10-20']

PRICE_mcdex['RETURN'] = (PRICE_mcdex['PRICE']/PRICE_mcdex['PRICE'].shift(1)) - 1
PRICE_mcdex['RETURN'] = PRICE_mcdex['RETURN']
PRICE_mcdex = PRICE_mcdex['2021-10-21':'2022-10-20']

MCAP_dydx = tt.get_mcap_circ(dydx)
MCAP_dydx = MCAP_dydx['2021-10-21':'2022-10-20']
MARKET_CAP_FULLY_DILUTED_dydx = tt.get_mcap_fully_diluted(dydx)
MARKET_CAP_FULLY_DILUTED_dydx = MARKET_CAP_FULLY_DILUTED_dydx['2021-10-21':'2022-10-20']

MCAP_gmx = tt.get_mcap_circ(gmx)
MCAP_gmx = MCAP_gmx['2021-10-21':'2022-10-20']
MARKET_CAP_FULLY_DILUTED_gmx = tt.get_mcap_fully_diluted(gmx)
MARKET_CAP_FULLY_DILUTED_gmx = MARKET_CAP_FULLY_DILUTED_gmx['2021-10-21':'2022-10-20']

MCAP_mcdex = tt.get_mcap_circ(mcdex)
MCAP_mcdex = MCAP_mcdex['2021-10-21':'2022-10-20']
MARKET_CAP_FULLY_DILUTED_mcdex = tt.get_mcap_fully_diluted(mcdex)
MARKET_CAP_FULLY_DILUTED_mcdex = MARKET_CAP_FULLY_DILUTED_mcdex['2021-10-21':'2022-10-20']

gmx_volume = pd.DataFrame(pd.read_csv('/content/Daily Trading Volume In The Past 365 Days. 2022-10-22 (1) gmx.csv'))
mcdex_volume = pd.DataFrame(pd.read_csv('/content/mcdex_bsc-trading-volume-1-year.csv'))
mcdex_volume = mcdex_volume.set_index('snapped_at')

VOLUME_dydx = tt.get_volume(dydx)
VOLUME_dydx = VOLUME_dydx['2021-10-21':'2022-10-20']
VOLUME_gmx = tt.get_volume(gmx)
VOLUME_gmx = VOLUME_gmx['2021-10-21':'2022-10-20']
VOLUME_mcdex = tt.get_volume(mcdex)
VOLUME_mcdex = VOLUME_mcdex['2021-10-21':'2022-10-20']
PRICE_dydx['PCT_CHANGE'] = PRICE_dydx['PRICE'].pct_change(1)

fee_dydx = pd.DataFrame(pd.read_csv('/content/Daily Fees In The Past 365 Days. 2022-10-03_dydx.csv'))
fee_dydx = fee_dydx.set_index('Date')
fee_gmx = pd.DataFrame(pd.read_csv('/content/Daily Fees Vs. Revenue In The Past 365 Days. 2022-10-22 gmx.csv'))
fee_gmx = fee_gmx.set_index('Date')
TVL_dydx = tt.get_tvl(dydx)
TVL_gmx = tt.get_tvl(gmx)
TVL_mcdex = tt.get_tvl(mcdex)
TVL_mcdex = TVL_mcdex['2021-10-21':'2022-10-20']
TVL_gmx = TVL_gmx['2021-10-21':'2022-10-20']
TVL_dydx = TVL_dydx['2021-10-21':'2022-10-20']

MARKET_CAP_FULLY_DILUTED_dydx[-1:]['market_cap_fully_diluted']

2022-10-20 00:00:00+00:00    1.502307e+09
Name: market_cap_fully_diluted, dtype: float64

In [7]:
# Annualized revenue for dydx

annualized_revenue = fee_gmx[-30:]['Revenue ($)'].mean() * 365

In [8]:
# Data for DYDX

def multiples_dydx(volume, TVL_dydx, PRICE_dydx, MCAP_dydx, MARKET_CAP_FULLY_DILUTED_dydx, current_price, market_cap, days):
    
    '''''''''
    name - unique id in our system
    current_price - most recent token price read from coingecko
    market_cap
    returns - token pct return to be used for calculating volatility
    days - how many days to annualize e.g. 30 would use most recent 30 days of revenue and annualize it
    users - pandas dataframe containing user data
    count - pandas dataframe containing transaction count data 
    '''''''''
    
    #volume, fees, revenue, tvl = data['Volume'], data['Fees'], data['Revenue'], data['TVL']
    volume = volume
    #fees = fee_dydx['Fees ($)']
    #revenue = fee_dydx['Fees ($)']
    tvl = TVL_dydx['tvl']
    #volume = volume[name]
    #fees = fees[name]
    #revenue = revenue[name]
    
    tvl = tvl.replace('NaN', np.nan)
    #tvl = tvl[name]
    
    daily_avg_volume = volume[-days:].mean()
    annualized_volume = daily_avg_volume * 365 
    annualized_fees = 7.396686e+07
    annualized_revenue = 7.396686e+07
    

    tvl_current = 3.684908e+08   
    tvl_turnover = volume[-days:].sum() / tvl[-days:].mean()

    
    price = current_price
    std = 78.5
    MC = 1.967311e+08    
    FDV = 1.502307e+09
    MC_AF = MC / annualized_fees
    FDV_AF = FDV / annualized_fees
    MC_TVL = MC / tvl_current
    FDV_TVL = FDV / tvl_current
    
    
    
    if annualized_revenue!=0:
        MC_AR = MC / annualized_revenue
        FDV_AR = FDV / annualized_revenue
    
    else:
        MC_AR = np.nan
        FDV_AR = np.nan
    
    titles = ['Daily Average Volume', 'Annualized Volume', 'Annualized Fees (AF)',
         'Annualized Revenue (AR)', 'TVL', 'TVL Turnover ' + str(30) + ' days', 'Token Price', 'Realized Volatility %', 'Market Cap (MC)', 'Fully Diluted Valuation (FDV)',
              'MC / AF', 'FDV / AF', 'MC / AR', 'FDV / AR', 
              'MC / TVL', 'FDV / TVL']
              
    data = [daily_avg_volume, annualized_volume, annualized_fees, 
            annualized_revenue, tvl_current, tvl_turnover, price, std, MC, FDV, 
            MC_AF, FDV_AF, MC_AR, FDV_AR, MC_TVL, FDV_TVL]
    
    return pd.DataFrame(data = data, index = titles, columns=['dydx'])

d = multiples_dydx(VOLUME_dydx['VOLUME'],TVL_dydx, PRICE_dydx['PRICE'], MCAP_dydx, MARKET_CAP_FULLY_DILUTED_dydx, 1.43, 188000000, 30)

In [9]:
# Data for GMX

def multiples_dydx(volume, TVL_gmx, PRICE_gmx, MCAP_gmx, MARKET_CAP_FULLY_DILUTED_gmx, current_price, market_cap, days):
    
    #volume, fees, revenue, tvl = data['Volume'], data['Fees'], data['Revenue'], data['TVL']
    volume = volume
    #fees = fee_dydx['Fees ($)']
    #revenue = fee_dydx['Fees ($)']
    tvl = TVL_gmx['tvl']
    #volume = volume[name]
    #fees = fees[name]
    #revenue = revenue[name]
    
    tvl = tvl.replace('NaN', np.nan)
    #tvl = tvl[name]
    
    daily_avg_volume = volume[-days:].mean()
    annualized_volume = daily_avg_volume * 365 
    annualized_fees = 94848317.095
    annualized_revenue = 28454495.043333333
    

    tvl_current = 4.593047e+08   
    tvl_turnover =  volume[-days:].sum() / tvl[-days:].mean()

    
    price = current_price
    std = 110.91761606086214
    MC = MCAP_gmx[-1:]['market_cap_circulating'][0]    
    FDV = MARKET_CAP_FULLY_DILUTED_gmx[-1:]['market_cap_fully_diluted'][0]
    MC_AF = MC / annualized_fees
    FDV_AF = FDV / annualized_fees
    MC_TVL = MC / tvl_current
    FDV_TVL = FDV / tvl_current
    
    
    
    if annualized_revenue!=0:
        MC_AR = MC / annualized_revenue
        FDV_AR = FDV / annualized_revenue
    
    else:
        MC_AR = np.nan
        FDV_AR = np.nan
    
    titles = ['Daily Average Volume', 'Annualized Volume', 'Annualized Fees (AF)',
         'Annualized Revenue (AR)', 'TVL', 'TVL Turnover ' + str(30) + ' days', 'Token Price', 'Realized Volatility %', 'Market Cap (MC)', 'Fully Diluted Valuation (FDV)',
              'MC / AF', 'FDV / AF', 'MC / AR', 'FDV / AR', 
              'MC / TVL', 'FDV / TVL']
              
    data = [daily_avg_volume, annualized_volume, annualized_fees, 
            annualized_revenue, tvl_current, tvl_turnover, price, std, MC, FDV, 
            MC_AF, FDV_AF, MC_AR, FDV_AR, MC_TVL, FDV_TVL]
    
    return pd.DataFrame(data = data, index = titles, columns=['gmx'])

g = multiples_dydx(gmx_volume['Trading volume ($)'],TVL_gmx, PRICE_gmx['PRICE'], MCAP_gmx, MARKET_CAP_FULLY_DILUTED_gmx, 34.9, 287000000, 30)

In [10]:
# Annualized volatility for GMX

np.std(PRICE_gmx['RETURN'][-30:]) * np.sqrt(365) * 100

110.91761606086214

In [11]:
# Data for MCDEX

def multiples_mcdex(volume, TVL_mcdex, PRICE_mcdex, MCAP_mcdex, MARKET_CAP_FULLY_DILUTED_mcdex, current_price, market_cap, days):
    
    #volume, fees, revenue, tvl = data['Volume'], data['Fees'], data['Revenue'], data['TVL']
    volume = volume
    #fees = fee_dydx['Fees ($)']
    #revenue = fee_dydx['Fees ($)']
    tvl = TVL_mcdex['tvl']
    #volume = volume[name]
    #fees = fees[name]
    #revenue = revenue[name]
    
    tvl = tvl.replace('NaN', np.nan)
    #tvl = tvl[name]
    
    daily_avg_volume = volume[-days:].mean()
    annualized_volume = daily_avg_volume * 365 
    annualized_fees = 986
    annualized_revenue = 490
    

    tvl_current = 1.076830e+07   
    tvl_turnover =  volume[-days:].sum() / tvl[-days:].mean()

    
    price = current_price
    std = 44.44155009087235
    MC = MCAP_mcdex[-1:]['market_cap_circulating'][0]    
    FDV = MARKET_CAP_FULLY_DILUTED_mcdex[-1:]['market_cap_fully_diluted'][0]
    MC_AF = MC / annualized_fees
    FDV_AF = FDV / annualized_fees
    MC_TVL = MC / tvl_current
    FDV_TVL = FDV / tvl_current
    
    
    
    if annualized_revenue!=0:
        MC_AR = MC / annualized_revenue
        FDV_AR = FDV / annualized_revenue
    
    else:
        MC_AR = np.nan
        FDV_AR = np.nan
    
    
    titles = ['Daily Average Volume', 'Annualized Volume', 'Annualized Fees (AF)',
         'Annualized Revenue (AR)', 'TVL', 'TVL Turnover ' + str(30) + ' days', 'Token Price', 'Realized Volatility %', 'Market Cap (MC)', 'Fully Diluted Valuation (FDV)',
              'MC / AF', 'FDV / AF', 'MC / AR', 'FDV / AR', 
              'MC / TVL', 'FDV / TVL']
              
    data = [daily_avg_volume, annualized_volume, annualized_fees, 
            annualized_revenue, tvl_current, tvl_turnover, price, std, MC, FDV, 
            MC_AF, FDV_AF, MC_AR, FDV_AR, MC_TVL, FDV_TVL]
    
    return pd.DataFrame(data = data, index = titles, columns=['mcdex'])

m = multiples_mcdex(VOLUME_mcdex['VOLUME'],TVL_mcdex, PRICE_mcdex['PRICE'], MCAP_mcdex, MARKET_CAP_FULLY_DILUTED_mcdex, 3.29, 1.583270e+07, 30)

In [12]:
# Annualized volatility for MCDEX

np.std(PRICE_mcdex['RETURN'][-30:]) * np.sqrt(365) * 100

44.44155009087235

In [78]:
# All data in one DataFrame

table = pd.concat([p,d,g,m],axis=1)

In [79]:
table.loc[:, "perpetual-protocol"] =table["perpetual-protocol"].map('{:,.2f}'.format)
table.loc[:, "dydx"] =table["dydx"].map('{:,.2f}'.format)
table.loc[:, "gmx"] =table["gmx"].map('{:,.2f}'.format)
table.loc[:, "mcdex"] =table["mcdex"].map('{:,.2f}'.format)

In [80]:
table

,perpetual-protocol,dydx,gmx,mcdex
Daily Average Volume,"16,049,852.81","579,438,976.79","165,791,739.91","4,510.03"
Annualized Volume,"5,858,196,274.19","211,495,226,526.97","60,513,985,067.39","1,646,162.26"
Annualized Fees (AF),"5,858,196.27","73,966,860.00","94,848,317.09",986.00
Annualized Revenue (AR),"976,366.05","73,966,860.00","28,454,495.04",490.00
TVL,"23,114,574.94","368,490,800.00","459,304,700.00","10,768,300.00"
TVL Turnover 30 days,21.13,44.15,11.16,0.01
Token Price,0.55,1.43,34.90,3.29
Realized Volatility %,86.08,78.50,110.92,44.44
Market Cap (MC),"38,649,796.43","196,731,100.00","286,627,806.57","15,832,697.30"
Fully Diluted Valuation (FDV),"82,337,048.51","1,502,307,000.00","467,014,116.94","32,978,513.17"
